Import Necessary Libraries

In [0]:
#Import Necessary Libraries 
import os
import json
!pip install pymongo  # Install the pymongo module
import pymongo
import pyspark.pandas as pd  # This uses Koalas that is included in PySpark version 3.2 or newer.
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DecimalType

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/672.1 kB ? eta -:--:--
     ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/672.1 kB 3.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 672.1/672.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/307.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 22.6 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


Instantiate Global Variables

In [0]:
#instantiate global variables
# Azure MySQL Server Connection Information ###################
jdbc_hostname = "<exy8eb-mysql>.mysql.database.azure.com"
jdbc_port = 3306
src_database = "underwear"

connection_properties = {
  "user" : "exy8eb",
  "password" : "password",
  "driver" : "org.mariadb.jdbc.Driver"
}

# MongoDB Atlas Connection Information ########################
atlas_cluster_name = "cluster_name.dfghbws"
atlas_database_name = "underwear"
atlas_user_name = "user_name"
atlas_password = "password"

# Data Files (JSON) Information ###############################
dst_database = "underwear"

base_dir = "dbfs:/FileStore/lab_data/retail/batch"
database_dir = f"{base_dir}/{dst_database}"

data_dir = f"{base_dir}/retail"
batch_dir = f"{data_dir}/batch"
stream_dir = f"{data_dir}/stream"

orders_stream_dir = f"{stream_dir}/orders"
inventory_transactions_stream_dir = f"{stream_dir}/inventory_transactions"
purhcase_orders_stream_dir = f"{stream_dir}/products"

orders_output_bronze = f"{database_dir}/fact_orders/bronze"
orders_output_silver = f"{database_dir}/fact_orders/silver"
orders_output_gold   = f"{database_dir}/fact_orders/gold"

inventory_trans_output_bronze = f"{database_dir}/fact_inventory_transactions/bronze"
inventory_trans_output_silver = f"{database_dir}/fact_inventory_transactions/silver"
inventory_trans_output_gold   = f"{database_dir}/fact_inventory_transactions/gold"

purchase_orders_output_bronze = f"{database_dir}/fact_purchase_orders/bronze"
purchase_orders_output_silver = f"{database_dir}/fact_purchase_orders/silver"
purchase_orders_output_gold   = f"{database_dir}/fact_purchase_orders/gold"

# Delete the Streaming Files ################################## 
dbutils.fs.rm(f"{database_dir}/fact_orders", True) 
dbutils.fs.rm(f"{database_dir}/fact_inventory_transactions", True) 
dbutils.fs.rm(f"{database_dir}/fact_purchase_orders", True) 

# Delete the Database Files ###################################
dbutils.fs.rm(database_dir, True)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-2926510672053708>, line 4
      2 import os
      3 import json
----> 4 import pymongo
      5 import pyspark.pandas as pd  # This uses Koalas that is included in PySpark version 3.2 or newer.
      6 from pyspark.sql.functions import col

ModuleNotFoundError: No module named 'pymongo'

Define Global Functions

In [0]:
#Define Global Functions
##################################################################################################################
# Use this Function to Fetch a DataFrame from the MongoDB Atlas database server Using PyMongo.
##################################################################################################################
def get_mongo_dataframe(user_id, pwd, cluster_name, db_name, collection, conditions, projection, sort):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    
    client = pymongo.MongoClient(mongo_uri)

    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    if conditions and projection and sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection).sort(sort)))
    elif conditions and projection and not sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection)))
    else:
        dframe = pd.DataFrame(list(db[collection].find()))

    client.close()
    
    return dframe

##################################################################################################################
# Use this Function to Create New Collections by Uploading JSON file(s) to the MongoDB Atlas server.
##################################################################################################################
def set_mongo_collection(user_id, pwd, cluster_name, db_name, src_file_path, json_files):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    client = pymongo.MongoClient(mongo_uri)
    db = client[db_name]
    
    '''Read in a JSON file, and Use It to Create a New Collection'''
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(src_file_path, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)

    client.close()
    
    return result

Fetch Date from Azure MySQL Database

In [0]:
%sql
DROP DATABASE IF EXISTS underwear_dlh CASCADE;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS underwear_dlh
COMMENT "DS-2002 Lab 06 Database"
LOCATION "dbfs:/FileStore/lab_data/underwear_dlh"
WITH DBPROPERTIES (contains_pii = true, purpose = "Final");

Get dim_date from Azure MySQL

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_date
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://exy8eb-mysql.mysql.database.azure.com:3306/underwear", --Replace with your Server Name
  dbtable "dim_date",
  user "exy8eb",    --Replace with your User Name
  password "password"  --Replace with you password
)

In [0]:
%sql
USE DATABASE underwear_dlh;

CREATE OR REPLACE TABLE underwear_dlh.dim_date
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/lab_data/underwear_dlh/dim_date"
AS SELECT * FROM view_date

In [0]:
%sql
DESCRIBE EXTENDED underwear_dlh.dim_date;

In [0]:
%sql
SELECT * FROM underwear_dlh.dim_date LIMIT 5

Get dim_products from AzureMySql

In [0]:
%sql
-- Create a Temporary View named "view_product" that extracts data from your MySQL Underwear database.
CREATE OR REPLACE TEMPORARY VIEW view_product
USING org.apache.sql.jdbc
OPTIONS (
  url "jdbc:mysql://exy8eb-mysql.mysql.database.azure.com:3306/underwear_dw2",
  dbtable "dim_products"
  user "exy8eb"
  password "password"
)

In [0]:
%sql
USE DATABASE underwear_dlh;

-- Create a new table named "northwind_dlh.dim_product" using data from the view named "view_product"
CREATE OR REPLACE TABLE underwear_dlh.dim_product
COMMENT "Products Dimension Table"
LOCATION "dbfs:/FileStore/lab_data/underwear_dlh/dim_product"
AS SELECT * FROM view_product

In [0]:
%sql
DESCRIBE EXTENDED underwear_dlh.dim_product;

In [0]:
%sql
SELECT * FROM underwear_dlh.dim_product LIMIT 5

Use PySparks to Read in dim_customers from csv

In [0]:
#read in csv
customer_csv = f"{batch_dir}/Underwear_DimCustomers.csv"

df_customers = spark.read.format('csv').options(header='true', inferSchema='true').load(customer_csv)
display(df_customers)

inventory_transactions_csv = f"{batch_dir}/Underwear_DimInventoryTransactions.csv"

df_inventory_transactions = spark.read.format('csv').options(header='true', inferSchema='true').load(inventory_transactions_csv)
display(df_inventory_transactions)

order_details_csv = f"{batch_dir}/Underwear_DimOrderDetails.csv"

df_order_details = spark.read.format('csv').options(header='true', inferSchema='true').load(order_details_csv)
display(df_order_details)

suppliers_csv = f"{batch_dir}/Underwear_DimSuppliers.csv"

df_suppliers = spark.read.format('csv').options(header='true', inferSchema='true').load(suppliers_csv)
display(df_suppliers)

In [0]:
df_customers.printSchema()
df_inventory_transactions.printSchema()
df_order_details.printSchema()

In [0]:
df_customers.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.dim_customers")
df_inventory_transactions.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.dim_inventory_transactions")
df_order_details.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.dim_order_details")
df_suppliers.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.dim_suppliers")

In [0]:
%sql
DESCRIBE EXTENDED underwear_dlh.dim_customers;
DESCRIBE EXTENDED underwear_dlh.dim_inventory_transactions;
DESCRIBE EXTENDED underwear_dlh.dim_order_details;
DESCRIBE EXTENDED underwear_dlh.dim_suppliers;

In [0]:
%sql
SELECT * FROM underwear_dlh.dim_customers LIMIT 5;
SELECT * FROM underwear_dlh.dim_inventory_transactions LIMIT 5;
SELECT * FROM underwear_dlh.dim_order_details LIMIT 5;
SELECT * FROM underwear_dlh.dim_suppliers LIMIT 5;

In [0]:
#verify dimension tables
%sql
USE underwear_dlh;
SHOW TABLES

Fetch Reference Data from a MongoDB Atlas Database

In [0]:
display(dbutils.fs.ls(batch_dir))  # '/dbfs/FileStore/lab_data/retail/batch'

Create a New MongoDB Database, and Load JSON Data Into a New MongoDB Collection

In [0]:
source_dir = '/dbfs/FileStore/lab_data/retail/batch'
json_files = {"order" : 'dim_order.json',
              "purchase_orders" : 'dim_purchase_orders.json',
              "employee" : 'dim_employee.json',
              "shipping_methods" : "dim_shipping_methods.json",
              }

set_mongo_collection(atlas_user_name, atlas_password, atlas_cluster_name, atlas_database_name, source_dir, json_files) 

In [0]:
%scala
import com.mongodb.spark._

val userName = "user_name"
val pwd = "password"
val clusterName = "cluster_name.xxxxx"
val atlas_uri = s"mongodb+srv://$userName:$pwd@$clusterName.mongodb.net/?retryWrites=true&w=majority"

In [0]:
%scala

val df_dim_order = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("uri", atlas_uri).load()
.option("database", "underwear")
.option("collection", "order")
.select("OrderId","CustomerId","EmployeeId","ShippingMethodId","OrderDate","ShipDate","FreightCharge")

display(df_dim_order)

val df_dim_purchase_order = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("uri", atlas_uri).load()
.option("database", "underwear")
.option("collection", "purchase_orders")
.select("PurchaseOrderId","SupplierId","EmployeeId","ShippingMethodId","OrderDate")

display(df_dim_purchase_orders)

val df_dim_employee = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("uri", atlas_uri).load()
.option("database", "underwear")
.option("collection", "employee")
.select("EmployeeId","EmployeeName")

display(df_dim_employee)

val df_dim_shipping_methods = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("uri", atlas_uri).load()
.option("database", "underwear")
.option("collection", "employee")
.select("ShippingMethodId","ShippingMethod")

display(df_dim_shipping_methods)

val df_dim_inventory_transactions = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("uri", atlas_uri).load()
.option("database", "underwear")
.option("collection", "inventory_transactions")
.select("TransactionId","ProductId", "PurchaseOrderId","MissingId","TransactionDate","UnitPurchasePrice","QuantityOrdered","QuantityReceived","QuantityMissing")

display(df_dim_inventory_transactions)

val df_dim_order_details = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("uri", atlas_uri).load()
.option("database", "underwear")
.option("collection", "order_details")
.select("OrderDetailId","OrderId", "ProductId","QuantitySold","UnitSalesPrice")

display(df_dim_order_details)

val df_dim_suppliers = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("uri", atlas_uri).load()
.option("database", "underwear")
.option("collection", "suppliers")
.select("SupplierId","SupplierName")

display(df_dim_suppliers)

In [0]:
%scala

df_dim_order.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.order")
df_dim_purchase_orders.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.purchase_orders")
df_dim_employee.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.employee")
df_dim_shipping_methods.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.shipping_methods")
df_dim_inventory_transactions.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.inventory_transactions")
df_dim_order_details.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.order_details")
df_dim_order_details.write.format("delta").mode("overwrite").saveAsTable("underwear_dlh.suppliers")

In [0]:
%sql

DESCRIBE EXTENDED underwear_dlh.order
DESCRIBE EXTENDED underwear_dlh.purchase_orders
DESCRIBE EXTENDED underwear_dlh.employee
DESCRIBE EXTENDED underwear_dlh.shipping_methods
DESCRIBE EXTENDED underwear_dlh.inventory_transactions
DESCRIBE EXTENDED underwear_dlh.order_details
DESCRIBE EXTENDED underwear_dlh.suppliers

Do Bronze Table to create fact_order table

In [0]:
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 #.option("cloudFiles.schemaHints", "fact_order_key BIGINT")
 #.option("cloudFiles.schemaHints", "OrderId BIGINT")
 #.option("cloudFiles.schemaHints", "CustomerId BIGINT")
 #.option("cloudFiles.schemaHints", "EmployeeId BIGINT") 
 #.option("cloudFiles.schemaHints", "ShippingMethodId BIGINT")
 #.option("cloudFiles.schemaHints", "OrderDate DECIMAL")
 #.option("cloudFiles.schemaHints", "ShipDate DECIMAL")
 #.option("cloudFiles.schemaHints", "FreightCharge DECIMAL")
 .option("cloudFiles.schemaLocation", products_output_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(products_stream_dir)
 .createOrReplaceTempView("products_raw_tempview"))

In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW orders_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM orders_raw_tempview
)

In [0]:
%sql
SELECT * FROM orders_bronze_tempview

In [0]:
(spark.table("orders_bronze_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{orders_output_bronze}/_checkpoint")
      .outputMode("append")
      .table("fact_orders_bronze"))

Create Silver Table

In [0]:
(spark.readStream
  .table("fact_orders_bronze")
  .createOrReplaceTempView("orders_silver_tempview"))

In [0]:
%sql
SELECT * FROM orders_silver_tempview

In [0]:
%sql
DESCRIBE EXTENDED orders_silver_tempview

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fact_orders_silver_tempview AS (
  SELECT o.fact_order_key,
      o.OrderId,
      od.OrderDetailId,
      od.ProductId,
      p.ProductName AS product_name,
      p.Color AS color,
      p.ModelDescription AS model_description, 
      p.FabricDescription AS fabric_description,
      p.Category AS category, 
      p.Gender AS gender,
      p.ProductLine AS product_line,
      p.Weight AS weight,
      p.Size AS size,
      p.PackSize AS pack_size,
      p.Status AS status, 
      p.InventoryDate AS inventory_date, 
      p.PurchasePrice AS purchase_price,
      o.EmployeeId,
      e.EmployeeName AS employee_name,
      o.ShippingMethodId,
      sm.ShippingMethod AS shipping_method,
      dd.OrderDate AS order_date,
      dd.month_name AS order_month_name,
      ds.ShipDate As ship_date,
      o.FreightCharge AS freight_charge,

  FROM orders_silver_tempview AS o
  INNER JOIN underwear_dlh.dim_employees AS e
  ON e.EmployeeId = o.EmployeeId
  INNER JOIN underwear_dlh.dim_customers AS c
  ON c.CustomerId = o.CustomerId
  INNER JOIN underwear_dlh.dim_shipping_methods AS sm
  ON sm.ShippingMethodId = o.ShippingMethodId
  INNER JOIN underwear_dlh.dim_order_details AS od
  ON od.OrderId = o.OrderId
  INNER JOIN underwear_dlh.dim_products AS p
  ON p.ProductId = o.ProductId
  LEFT OUTER JOIN underwear_dlh.dim_date AS dd
  ON od.DateId = o.OrderDate
  LEFT OUTER JOIN underwear_dlh.dim_date AS ds
  ON ds.DateId = o.ShipDate
)

In [0]:
(spark.table("fact_orders_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{orders_output_silver}/_checkpoint")
      .outputMode("append")
      .table("fact_orders_silver"))

In [0]:
%sql
SELECT * FROM fact_orders_silver

In [0]:
%sql
DESCRIBE EXTENDED underwear_dlh.fact_orders_silver

Create Gold Tables

In [0]:
#gold aggregations
%sql
CREATE OR REPLACE TABLE underwear_dlh.fact_monthly_orders_by_model_description_gold AS (
  SELECT ProductId AS ProductID
    , model_description AS ModelDescription
    , order_month_name AS OrderMonth
    , COUNT(ProductId) AS ProductCount
  FROM underwear_dlh.fact_orders_silver
  GROUP BY ProductId, ModelDescription, OrderMonth
  ORDER BY ProductCount DESC);

SELECT * FROM underwear_dlh.fact_monthly_products_by_customer_gold;

Create Bronze Table to Create an Inventory Transactions Fact Table


In [0]:
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 #.option("cloudFiles.schemaHints", "TransactionId BIGINT")
 #.option("cloudFiles.schemaHints", "ProductId BIGINT")
 #.option("cloudFiles.schemaHints", "PurchaseOrderId BIGINT")
 #.option("cloudFiles.schemaHints", "MissingId BIGINT") 
 #.option("cloudFiles.schemaHints", "TransactionDate BIGINT")
 #.option("cloudFiles.schemaHints", "UnitPurchasePrice DECIMAL")
 #.option("cloudFiles.schemaHints", "QuantityOrdered DECIMAL")
 #.option("cloudFiles.schemaHints", "QuantityReceived DECIMAL")
 #.option("cloudFiles.schemaHints", "QuantityMissing DECIMAL") 
 .option("cloudFiles.schemaLocation", inventory_trans_output_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(inventory_trans_stream_dir)
 .createOrReplaceTempView("inventory_transactions_raw_tempview"))

In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW inventory_transactions_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM inventory_transactions_raw_tempview
)

In [0]:
%sql
SELECT * FROM inventory_transactions_raw_tempview

In [0]:
(spark.table("inventory_transactions_bronze_tempview")
     .writeStream
     .format("delta")
     .option("checkpointLocation", f"{inventory_trans_output_bronze}/_checkpoint")
     .outputMode("append")
     .table("fact_inventory_transactions_bronze"))

Create Silver Table

In [0]:
(spark.readStream
  .table("fact_inventory_transactions_bronze")
  .createOrReplaceTempView("inventory_transactions_silver_tempview"))

In [0]:
%sql
SELECT * FROM inventory_transactions_silver_tempview

In [0]:
%sql
DESCRIBE EXTENDED inventory_transactions_silver_tempview

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fact_inventory_transactions_silver_tempview AS (
  SELECT it.fact_inventory_transaction_key,
      it.TransactionId,
      it.ProductId,
      p.ProductName AS product_name,
      p.Color AS color, 
      p.ModelDescription AS model_description, 
      p.FabricDescription AS fabric_description, 
      p.Category AS category, 
      p.Gender AS gender, 
      p.ProductLine AS product_line, 
      p.Weight AS weight, 
      p.Size AS size, 
      p.PackSize AS pack_size, 
      p.Status AS status, 
      p.InventoryDate AS inventory_date, 
      p.PurchasePrice AS purchase_price, 
      it.PurchaseOrderId, 
      po.SupplierId, 
      s.SupplierName AS supplier_name, 
      po.EmployeeId, 
      e.EmployeeName AS employee_name, 
      po.ShippingMethodId, 
      sm.ShippingMethod AS shipping_method, 
      po.OrderDate AS order_date,
      it.MissingId,
      it.TransactionDate AS transaction_date,
      it.UnitPurchasePrice AS unit_purchase_price, 
      it.QuantityOrdered AS quantity_ordered, 
      it.QuantityReceived AS quantity_received, 
      it.QuantityMissing AS quantity_missing,
      cd.day_name_of_week AS created_day_name_of_week,
      cd.day_of_month AS created_day_of_month,
      cd.weekday_weekend AS created_weekday_weekend,
      cd.month_name AS created_month_name,
      cd.calendar_quarter AS created_quarter, 
      cd.calendar_year AS created_year, 
  FROM inventory_transactions_silver_tempview AS it
  INNER JOIN underwear_dlh.dim_product AS p
  ON p.ProductId = it.ProductId
  INNER JOIN northwind_dlh.dim_purchase_orders AS po
  ON po.PurchaseOrderId = it.PurchaseOrderId
  INNER JOIN northwind_dlh.dim_employee AS e
  ON e.EmployeeId = it.EmployeeId
  LEFT OUTER JOIN northwind_dlh.dim_shipping_methods AS sm
  ON sm.ShippingMethodId = it.ShippingMethodId
  LEFT OUTER JOIN northwind_dlh.dim_date AS cd
  ON cd.DateId = it.TransactionDate
)

In [0]:
(spark.table("fact_inventory_transactions_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{inventory_trans_output_silver}/_checkpoint")
      .outputMode("append")
      .table("fact_inventory_transactions_silver"))

In [0]:
%sql
SELECT * FROM fact_inventory_transactions_silver

In [0]:
%sql
DESCRIBE EXTENDED fact_inventory_transactions_silver

Gold Aggregations

In [0]:
%sql
-- Author a query that returns the Total Quantity grouped by the Quarter Created, Inventory Transaction Type, and Product
-- Sort by the Total Quantity Descending
CREATE OR REPLACE TABLE underwear_dlh.fact_inventory_transactions_gold AS (
  SELECT created_quarter AS QuarterCreated
    , InventoryTransactionId AS Transactions
    , product_name AS Product
    , SUM(quantity_ordered) AS TotalQuantityOrdered
  FROM underwear_dlh.fact_inventory_transactions_silver
  GROUP BY QuarterCreated, Transaction, Product
  ORDER BY TotalQuantity DESC);

SELECT * FROM underwear_dlh.fact_inventory_transactions_gold;

Make Bronze Table to create a Purchase Orders Fact Table

In [0]:
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 #.option("cloudFiles.schemaHints", "fact_order_key BIGINT")
 #.option("cloudFiles.schemaHints", "PurchaseOrderId BIGINT")
 #.option("cloudFiles.schemaHints", "SuppliedId BIGINT")
 #.option("cloudFiles.schemaHints", "EmployeeId BIGINT") 
 #.option("cloudFiles.schemaHints", "ShippingMethodId BIGINT")
 #.option("cloudFiles.schemaHints", "OrderDate DECIMAL")
 .option("cloudFiles.schemaLocation", purchase_orders_output_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(purchase_orders_stream_dir)
 .createOrReplaceTempView("purchase_orders_raw_tempview"))

In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW purchase_orders_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM purchase_orders_raw_tempview
)

In [0]:
%sql
SELECT * FROM purchase_orders_bronze_tempview

In [0]:
(spark.table("purchase_orders_bronze_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{purchase_orders_output_bronze}/_checkpoint")
      .outputMode("append")
      .table("fact_purchase_orders_bronze"))

Create Silver Tables

In [0]:
(spark.readStream
  .table("fact_purchase_orders_bronze")
  .createOrReplaceTempView("purchase_orders_silver_tempview"))

In [0]:
%sql
SELECT * FROM purchase_orders_silver_tempview

In [0]:
%sql
DESCRIBE EXTENDED purchase_orders_silver_tempview

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fact_orders_silver_tempview AS (
  SELECT po.fact_order_key,
      po.PurchaseOrderId,
      po.SupplierId,
      s.SupplierName AS supplier_name,
      po.EmployeeId, 
      e.EmployeeName AS employee_name, 
      po.ShippingMethodId,
      sm.ShippingMethod AS shipping_method, 
      po.OrderDate,
      od.day_name_of_week AS creation_day_name_of_week, 
      od.day_of_month AS creation_day_of_month, 
      od.weekday_weekend AS creation_weekday_weekend, 
      od.month_name AS creation_month_name,
      od.calendar_quarter AS creation_quarter, 
      od.calendar_year AS creation_year, 
  FROM purchase_orders_silver_tempview AS po
  INNER JOIN northwind_dlh.dim_supplier AS s
  ON s.SupplierId = po.SupplierId
  INNER JOIN northwind_dlh.dim_employee AS e
  ON p.EmployeeId = po.EmployeeId
  INNER JOIN northwind_dlh.dim_shipping_method AS sm
  ON ce.ShippingMethodId = po.ShippingMethodId
  LEFT OUTER JOIN northwind_dlh.dim_date AS od
  ON od.DateId = po.OrderDate
)


In [0]:
(spark.table("fact_purchase_orders_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{purchase_orders_output_silver}/_checkpoint")
      .outputMode("append")
      .table("fact_purchase_orders_silver"))

In [0]:
%sql
SELECT * FROM fact_purchase_orders_silver

In [0]:
%sql
DESCRIBE EXTENDED fact_purchase_orders_silver

Gold Tables - Aggregations

In [0]:
%sql
-- Author a query that returns the Total List Price grouped by Supplier and Product and sorted by Total List Price descending.
CREATE OR REPLACE TABLE underwear_dlh.fact_order_purchase_orders_by_shipping_method_gold AS (
  SELECT supplier_name AS Supplier
    , product_name AS Product
    , SUM(shipping_method) AS ShippingMethod
  FROM underwear_dlh.fact_orders_silver
  GROUP BY Supplier, Product
  ORDER BY ShippingMethod DESC);

SELECT * FROM underwear_dlh.fact_order_purchase_orders_by_shipping_method_gold;

Clean Up the File System

In [0]:
%fs rm -r /FileStore/lab_data/